# Template für Topic Modeling
Dieses Template soll dabei helfen, Topic Modeling automatisiert und einheitlich durchzuführen. 

## Allgemeine Vorbereitungsschritte

In [1]:
# Aktuelles Arbeitsverzeichnis anzeigen und bei Bedarf anpassen
# print(os.getcwd())
# os.chdir("C:/SV/HEX/Topic Modeling")

### Pakete laden

In [2]:
# Aktuelles Arbeitsverzeichnis anzeigen und bei Bedarf anpassen
import os
print(os.getcwd())
os.chdir("C:/Users/mhu/Documents/github/topic_model_it/")
#os.chdir("C:/Users/mhu/Documents/github/HEX_topic_model_it/")

import pandas as pd
import stopwords
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
import sklearn
import re
import spacy
import numpy as np
import random
import torch
from bertopic.vectorizers import ClassTfidfTransformer 
import openpyxl
import optuna
from sklearn.cluster import KMeans
from bertopic.representation import MaximalMarginalRelevance

model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2", device="cuda" if torch.cuda.is_available() else "cpu")
print("CUDA available:", torch.cuda.is_available())
print("Current device:", model.device)

c:\Users\mhu\Documents\github\topic_model_it\notebooks\model_1


c:\Users\mhu\miniconda3\envs\bertopic-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CUDA available: True
Current device: cuda:0


### Seed setzen
Wir setzen einen festen Seed, um Zufallszahlen in NumPy und PyTorch reproduzierbar zu machen, sowohl auf der CPU als auch auf der GPU (falls verfügbar). Das stellt sicher, dass Berechnungen mit zufälligen Operationen bei wiederholter Ausführung dieselben Ergebnisse liefern.

In [3]:
seed = 40  # Initialisiert den Seed-Wert für reproduzierbare Ergebnisse
np.random.seed(seed)  # Setzt den Seed für NumPy-Zufallszahlengeneratoren
random.seed(seed)  # Setzt den Seed für den Python-eigenen Zufallszahlengenerator
torch.manual_seed(seed)  # Setzt den Seed für PyTorch-Zufallszahlen
if torch.cuda.is_available():  # Überprüft, ob CUDA (GPU-Unterstützung) verfügbar ist
    torch.cuda.manual_seed_all(seed)  # Setzt den Seed für alle CUDA-Zufallszahlen (für GPU-Berechnungen)

### Datensätze einlesen
Der Trainings- und der Test-Datensatz werden hier eingelesen. Als Faustregel gilt, der Trainingsdatensatz sollte 80% und der Test-Datensatz 20% des Volumens ausmachen. 
Der Trainings-Datensatz wird für das trainieren / fitten des Modells verwendet. Der Test-Datensatz beinhaltet eine (in diesem Fall manuell erstellte) sogenannte "Ground Truth". Dies ist der Goldstandard, anhand dessen das Modell auf Performance hin überprüft wird. 

In [4]:
# Training-Datensatz
training_set = pd.read_csv("data/informatikkurse.csv")  # Liest die CSV-Datei ein und speichert sie in einem DataFrame
# training_set = training_set.sample(n=500, random_state=42)  # Zieht eine Zufallsstichprobe von 500 Zeilen aus dem DataFrame mit festgelegtem Seed für Reproduzierbarkeit
training_set = training_set.apply(lambda x: x.fillna('') if x.dtype == 'O' else x)  # Ersetzt fehlende Werte durch leere Strings in Objektspalten (Strings) und belässt numerische Spalten unverändert
training_set['titel_kursbesch'] = training_set['veranstaltung_titel'] + ' ' + training_set['kursbeschreibung']  # Kombiniert die Spalten "titel" und "kursbeschreibung" zu einer neuen Spalte "titel_kursbesch"
docs = training_set['titel_kursbesch'].tolist()  # Konvertiert die Inhalte der Spalte "titel_kursbesch" in eine Liste von Strings

C:\Users\mhu\AppData\Local\Temp\ipykernel_34904\3557994817.py:2: DtypeWarning: Columns (15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  training_set = pd.read_csv("data/informatikkurse.csv")  # Liest die CSV-Datei ein und speichert sie in einem DataFrame


In [13]:
import pandas as pd

# 1. Daten laden und direkt alle NaNs durch leere Strings ersetzen
df = pd.read_csv("data/labelled_data_final.csv", sep=";").fillna('')

# 2. Titel und Beschreibung kombinieren
df['titel_kursbesch'] = df['veranstaltung_titel'] + ' ' + df['kursbeschreibung']

# 3. Filtern: Behalte nur Zeilen, in denen "Synonyme" nicht leer ist
# (Entfernt Zeilen, die nach dem strip() leer wären)
df = df[df["Synonyme"].str.strip() != ""].copy()

# 4. Ergebnisse extrahieren
test_set = df['titel_kursbesch'].reset_index(drop=True)
ground_truth = df["Synonyme"].str.lower().str.strip().tolist()

In [14]:
ground_truth

['software development, software design, software architecture, application development, software engineering, softwareentwicklung, softwaredesign, softwarearchitektur, anwendungsentwicklung',
 'theory of computation, computational theory, formal methods, automata theory, theoretische informatik, berechenbarkeitstheorie, computertheorie, formale methoden, automatentheorie',
 'robot systems, autonomous robots, robot engineering, robot programming, robotik, robotersysteme, autonome roboter, robotertechnik, roboterprogrammierung',
 'computational biology, genomic analysis, biological data analysis, bioinformatik, computergestützte biologie, genomanalyse, biologische datenanalyse',
 'big data analytics, data analytics, data mining, data engineering, data science und big data, big-data-analyse, datenanalyse, data mining, datenengineering',
 'vr, extended reality, immersive technologies, virtuelle und erweiterte realität, vr, extended reality, immersive technologien',
 'embedded computing, r

## NLP Vorbereitungsschritte
Zunächst werden die Trainingsdaten eingelesen und die gängigen Vorbereitungsschritte für NLP durchgeführt. Diese wären:
* Stopwords entfernen
* CountVectorizer spezifizieren

### Stopwords entfernen
Im Kontext des hier zu modellierenden Topic Modells werden sowohl standardisierte englische, deutsche als auch individuelle Stopwords generiert und im Objekt `sw` zusammengespielt.
Die Stopwords können je nach Anwendungsfall ergänzt oder reduziert werden.

In [ ]:
from utils import stopwords_config

irrelevant_terms = stopwords_config.irrelevant_terms

sw = list(stopwords.get_stopwords("en"))
sw.extend(list(stopwords.get_stopwords("de")))
sw.extend(irrelevant_terms) 

['vl',
 'übung',
 'übungen',
 'seminar',
 'arbeitsgruppenseminar',
 'oberseminar',
 'proseminar',
 'blockveranstaltung',
 'vorlesung',
 'kolloquium',
 'theoriekolloquium',
 'einführung',
 'tutorium',
 'ue',
 'vereinbarung',
 'projekt',
 'praktikum',
 'masterprojekt',
 'wiederholerklausur',
 'fortgeschrittenenpraktikum',
 'hauptseminar',
 'fachpraktikum',
 'ergänzungsvorlesung',
 'forschungspraktikum',
 'begleitseminar',
 'abschlussarbeiten',
 'unterrichtspraktikum',
 'masterseminar',
 'proseminare',
 'praxisseminar',
 'praxissemester',
 'schulpraxis',
 'ringpraktikum',
 'basispraktikum',
 'praxistage',
 'industriepraktikum',
 'vorkurs',
 'projektseminar',
 'juniorprofessur',
 'masterarbeiten',
 'forschungsseminar',
 'modulbeschreibung',
 'veranstaltung',
 'kommentare',
 'raum',
 'uhrzeit',
 'vereinbarung',
 'vorlesung',
 'praktikum',
 'masterprojekt',
 'wiederholerklausur',
 'fortgeschrittenenpraktikum',
 'hauptseminar',
 'fachpraktikum',
 'ergänzungsvorlesung',
 'forschungspraktikum',

### Lemmatisierung
Durch Lemmatisierung werden die Wörter in einheitliche Begriffe umgewandelt, sodass diese robuster werden. 

## Anwendung: Konfiguration, Training und Evaluation des Topic Models
Hier muss alles innerhalb einer einzigen Code-Zelle erfolgen, da bei allen Konfigurationen variable Parameter vorkommen und wir diese durch das optuna-Package optimieren wollen.

In [16]:
# Im Objective werden die verschiedenen Parameter-Settings gesetzt, über welche man optimieren möchte
def objective(trial):

  try:
    # Embedding Settings
    embedding_model_name = trial.suggest_categorical("embedding_model", ["paraphrase-multilingual-mpnet-base-v2", "paraphrase-multilingual-MiniLM-L12-v2"])
    # UMAP Settings
    n_neighbors = trial.suggest_categorical("n_neighbors", [15, 30, 50])
    min_dist = 0.0
    n_components = trial.suggest_categorical("n_components", [5, 7, 10])
    # HDBSCAN Settings
    min_cluster_size = trial.suggest_int("min_cluster_size", 10, 100) # Statt [80, 320, 480]    #min_samples = trial.suggest_int("min_samples", 3, 10, 1)
    #min_samples = trial.suggest_int("min_samples", 3, 10, 1)
    # BERTopic Settings
    #nr_topics = trial.suggest_categorical("nr_topics", [5, 6, 7, 8, 9, 10, 15])
    diversity = trial.suggest_float("diversity", 0.0, 0.2, step = 0.1)


    #--------------------------------------------------------------------------------------------------------------------------------------------
    # Konfiguration
    #--------------------------------------------------------------------------------------------------------------------------------------------

    # CountVectorizer
    vectorizer = CountVectorizer(
      stop_words=irrelevant_terms,  # Entfernt Stopwörter basierend auf der angegebenen Liste (sw)
      token_pattern=r'\b\w+\b',  # Extrahiert nur ganze Wörter, d. h. keine Sonderzeichen oder Zahlen
      ngram_range=(1, 3)  # Erstellt 1-Gramme (einzelne Wörter) bis 3-Gramme (Wortgruppen aus bis zu 3 aufeinanderfolgenden Wörtern)
    )

    # Embedding Settings  
    embedding_model = SentenceTransformer(embedding_model_name)
    
    # UMAP Settings
    umap_model = UMAP(n_neighbors=n_neighbors, min_dist=min_dist, n_components=n_components, metric="cosine", random_state=13)

    # HDBSCAN Settings
    hdbscan_model = HDBSCAN(min_cluster_size=min_cluster_size, gen_min_span_tree=True, prediction_data=True)

    # Representation Settings
    representation_model = MaximalMarginalRelevance(diversity=diversity)


    #--------------------------------------------------------------------------------------------------------------------------------------------
    # Training
    #--------------------------------------------------------------------------------------------------------------------------------------------

    # BERTopic initialisieren
    topic_model = BERTopic(
      embedding_model=embedding_model,
      #min_topic_size=10,
      #nr_topics=nr_topics, 
      language="multilingual",
      umap_model=umap_model,
      vectorizer_model=vectorizer,
      hdbscan_model=hdbscan_model,
      top_n_words = 30,
      representation_model=representation_model
    )

    # BERTopic trainieren
    topic_model_quanten = topic_model.fit(docs)


    #--------------------------------------------------------------------------------------------------------------------------------------------
    # Evaluation
    #--------------------------------------------------------------------------------------------------------------------------------------------

    # BERTopic auf Test-Daten anwenden
    topics, probs = topic_model_quanten.transform(test_set)
    print(topic_model_quanten.get_topic_freq())

    # Outlier reduzieren
    topics = topic_model_quanten.reduce_outliers(test_set, topics)

    # Resultierende Topic-Nummern mit den Representations (= relevante Begriffe) zu einem Datensatz kombinieren
    dataframe_with_results_left = pd.DataFrame(topics, columns = ["Topic"])
    dataframe_with_results_right = pd.DataFrame(topic_model_quanten.get_topic_info().set_index('Topic')[['Representation']])
    dataframe_with_results = dataframe_with_results_left.join(dataframe_with_results_right, on="Topic")

    # Goldstandard (Ground Truth) mit den Ergebnissen abgleichen und Score berechnen (Score = Anteil korrekter Topic-Zuweisungen)
    row_number = 0
    metric = 0
    while row_number < len(ground_truth):
      # Den Goldstandard in eine Liste von Keywords umwandeln
      ground_truth_current_iteration = ground_truth[row_number].split(", ")
      result_current_iteration = dataframe_with_results.at[row_number, "Representation"]

      # Überprüfen, ob irgendein Begriff aus dem Resultat im Goldstandard zum Text vorkommt (1 = ja, 0 = nein)
      if any(element in result_current_iteration for element in ground_truth_current_iteration):
              metric += 1
      else: metric += 0

      row_number = row_number+1

      print(result_current_iteration)
      print(ground_truth_current_iteration)
      print("---------------------------------------------------------------------------------------------------------------")

    metric_score = metric/row_number


    #--------------------------------------------------------------------------------------------------------------------------------------------
    # Ergebnis printen und Score returnen
    #--------------------------------------------------------------------------------------------------------------------------------------------

    # Ergebnis printen
    #print("Modell evaluiert mit einem Score von ", metric_score, ". " \
    #"Verwendete Parameter: embedding model: ", embedding_model_name, ", nr_topics: ", nr_topics, ", n_neighbors: ", n_neighbors, ", min_dist: ", min_dist,
    #", n_components: ", n_components, ", min_cluster_size: ", min_cluster_size)
  
    return metric_score 
  
  except Exception as e:
      print("Trial wird aufgrund eines Errors übersprungen")
      print("Verwendete Parameter: embedding model: ", embedding_model_name, ", n_neighbors: ", n_neighbors, ", min_dist: ", min_dist,\
      ", n_components: ", n_components, ", min_cluster_size: ", min_cluster_size)
      print(e)
      raise optuna.TrialPruned()

In [ ]:
study = optuna.create_study(direction = "maximize")
study.optimize(objective, n_trials = 100)

print("Best parameters:", study.best_params)

[I 2026-02-24 16:37:10,829] A new study created in memory with name: no-name-16bf1ef7-f786-473b-a164-9c0c2ea546c6
[I 2026-02-24 16:38:31,214] Trial 0 finished with value: 0.15666666666666668 and parameters: {'embedding_model': 'paraphrase-multilingual-MiniLM-L12-v2', 'n_neighbors': 15, 'n_components': 5, 'min_cluster_size': 49, 'diversity': 0.2}. Best is trial 0 with value: 0.15666666666666668.


     Topic  Count
4       -1  21689
21       0    853
24       1    651
47       2    596
10       3    500
..     ...    ...
209    278     50
42     279     50
41     280     50
25     281     50
155    282     49

[284 rows x 2 columns]
['b software engineering', 'b software', 'foundations of software', 'b foundations of', 'b foundations', 'swl b software', 'software engineering lab', 'software engineering', 'fse b foundations', 'of software']
['software development', 'software design', 'software architecture', 'application development', 'software engineering', 'softwareentwicklung', 'softwaredesign', 'softwarearchitektur', 'anwendungsentwicklung']
---------------------------------------------------------------------------------------------------------------
['mathematics', 'logik mathematische', 'logik mathematische logik', 'mathematische logik mathematische', 'mathematische logik', 'campus heilbronn wihn0001', 'computational mathematics', 'kombinatorik aussagenlogik algebraische',

In [ ]:
print("Best parameters:", study.best_params)